In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import ExcelWriter #este va servir para exportar achivos a excel
import xlrd

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [6]:
#para encontrar una grafica de excel y ponerla en python 
datos = "Processed_Images_T2.xlsx" #busco el archivo y lo pego con el indicativo por ejemplo .xlsx
df = pd.read_excel(datos, sheet_name="T2")#sacar del excel a python
df.drop(columns= ["#","PHOTO"], inplace=True) #para borrar una columna que no queremos
df.sort_values(by="DEPTH")
#print(df.shape) #para saber el tamaño de como quedo antes de borrar los valore nulos
df.dropna(inplace=True)
df.set_index(np.array(range(0,(df.shape)[0])), inplace=True)
print(df.shape) #para saber el tamaño de como quedo despues de borrar 
#df.info()
df.head()

(48000, 2)


,DEPTH,GRAY
0,3671.000000,28.6000
1,3671.001875,27.9250
2,3671.003750,27.3750
3,3671.005625,28.4625
4,3671.007500,29.5625


In [17]:
gray = np.array(df.loc[:,'GRAY']) # crea un vector nuev0 todas las filas que tenga la columna GRAY
cutoff = np.zeros(gray.shape) 
# hacemos el condicional
cut_off = 250
for i in range (gray.shape[0]):
  if (gray[i] >= 80):
    cutoff[i] = 1

df['cutoff'] = cutoff
df["cutoff"].value_counts() #esto sirve para saber cuantos valores de 1 y de 0 que hay en el cutoff hay 
#df.head()

1.0    25332
0.0    22668
Name: cutoff, dtype: int64

In [18]:
#Definición de Vector de Salida y Partición en Train y Test Stratified
from sklearn.model_selection import StratifiedShuffleSplit #la dicision de 70 y 30 halla quede propocional los valores de salida de 100% proporsional en ambos parametros de porcentaje
from sklearn.preprocessing import StandardScaler #esto sirve para normalizar los datos osea sacar el max y el minimo yy dividir todos los valores por el maximo para todas las columnas queden en un ranco de 0 a 1

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)#tamaño de prueba
for train_index, test_index in split.split(df, df["cutoff"].values):
    Xtrain_strat = df.loc[train_index]
    Xtest_strat = df.loc[test_index]

ytrain_strat = np.array(Xtrain_strat['cutoff'])
ytest_strat = np.array(Xtest_strat['cutoff'])
Xtrain_strat.drop(columns='cutoff', inplace=True)
Xtest_strat.drop(columns='cutoff', inplace=True)

#Normalización de los datos
normalizar = StandardScaler()
Xtest_strat = normalizar.fit_transform(Xtest_strat)

print("Tamaño de Entrenamiento: ",Xtrain_strat.shape[0])
print("Tamaño de Prueba: ",Xtest_strat.shape[0])

Tamaño de Entrenamiento:  33600
Tamaño de Prueba:  14400


In [19]:
#Definición de Pasos y Parámetros
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV,cross_val_score, cross_val_predict 
from sklearn.metrics import mean_absolute_error, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
import numpy as np
import joblib
# pasos que debe hacer con los datos #pca es un algoritmo para hacer reduccion de dimecion 
steps=[[('norm', StandardScaler()),
        ('rep', PCA()),
       ('class', RandomForestClassifier())],
       [('norm', StandardScaler()),
        ('class', RandomForestClassifier())],
       [('norm', StandardScaler()),
        ('rep', PCA()),
        ('class', SVC())],
       [('norm', StandardScaler()),
        ('class', SVC())],]
# con que parametros quiero que haga lo que esta arriba
parameters = [{'rep__n_components':[2,4,7],
              'class__max_depth': [8,10,12,15],
              'class__n_estimators' : [20,40,60,80,100]
              }, 
              {'class__max_depth': [8,10,12,15],
              'class__n_estimators' : [20,40,60,80,100]
              },
              {'rep__n_components':[2,4,6,7],
              'class__C': [1,20,50,100,150],
              'class__gamma': [0.01, 0.001, 0.0001]
              },
              {'class__C': [1,20,50,100,150],
               'class__gamma': [0.01, 0.001, 0.0001]
              }]

cross_scores_strat = {} #puntaje de precision
err_strat = []
conf_matrxs_strat = []
best_model_strat = []
filename_strat = 'modelos_class'#'resultados/people_comp'
tests_strat = ['PCA - RandomForest','Random Forest','PCA - SVC','SVC']

for i in range (len(steps)):
    pipeline = Pipeline(steps = steps[i])

    grid_search = GridSearchCV(pipeline, parameters[i], n_jobs=-1,cv=3,
                               scoring='balanced_accuracy',verbose=5) #balanceado
    
    print("\nEntrenando: {} \n".format(tests_strat[i]))
    grid_search.fit(Xtrain_strat, ytrain_strat)
    
    #mejor modelo entrenado
    best_model_n = grid_search.best_estimator_
    best_model_strat += [grid_search.best_estimator_]
    joblib.dump(best_model_strat,filename_strat+".pkl")
    print("\nMejores parámetros con {} : {}".format(tests_strat[i],grid_search.best_params_))
    
    #evaluación:
    ytest_pred = cross_val_predict(best_model_n, Xtest_strat, ytest_strat, cv=3)
    cross_score = np.mean(cross_val_score(best_model_n, Xtest_strat, ytest_strat, cv=3, scoring="balanced_accuracy"))
    cross_scores_strat[tests_strat[i]] = cross_score
    conf_matrx = confusion_matrix(ytest_strat, ytest_pred, normalize= 'true')

    print("\nCros-Score con {} : {:.3f}\n".format(tests_strat[i],cross_score))
    err_strat.append(cross_score)
    conf_matrxs_strat.append(conf_matrx)


Entrenando: PCA - RandomForest 

Fitting 3 folds for each of 60 candidates, totalling 180 fits


C:\Users\User\anaconda3\envs\VCThesis\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.99731485        nan        nan 0.99713905        nan        nan
 0.9971258         nan        nan 0.99737456        nan        nan
 0.99726508        nan        nan 0.99758685        nan        nan
 0.99790363        nan        nan 0.99777426        nan        nan
 0.99764654        nan        nan 0.99775933        nan        nan
 0.99774104        nan        nan 0.99761835        nan        nan
 0.99785053        nan        nan 0.99794506        nan        nan
 0.99778419        nan        nan 0.99762497        nan        nan
 0.99797656        nan        nan 0.9976117         nan        nan
 0.99782565        nan        nan 0.99788536        nan        nan]
  warnings.warn(



Mejores parámetros con PCA - RandomForest : {'class__max_depth': 15, 'class__n_estimators': 40, 'rep__n_components': 2}

Cros-Score con PCA - RandomForest : 0.996


Entrenando: Random Forest 

Fitting 3 folds for each of 20 candidates, totalling 60 fits

Mejores parámetros con Random Forest : {'class__max_depth': 8, 'class__n_estimators': 20}

Cros-Score con Random Forest : 1.000


Entrenando: PCA - SVC 

Fitting 3 folds for each of 60 candidates, totalling 180 fits


C:\Users\User\anaconda3\envs\VCThesis\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.99420213        nan        nan        nan 0.99354045        nan
        nan        nan 0.9781867         nan        nan        nan
 0.99918405        nan        nan        nan 0.99647081        nan
        nan        nan 0.9910196         nan        nan        nan
 0.99891039        nan        nan        nan 0.99804633        nan
        nan        nan 0.99300475        nan        nan        nan
 0.99909942        nan        nan        nan 0.99886561        nan
        nan        nan 0.99426514        nan        nan        nan
 0.99892361        nan        nan        nan 0.99899166        nan
        nan        nan 0.99552549        nan        nan        nan]
  warnings.warn(



Mejores parámetros con PCA - SVC : {'class__C': 20, 'class__gamma': 0.01, 'rep__n_components': 2}

Cros-Score con PCA - SVC : 0.998


Entrenando: SVC 

Fitting 3 folds for each of 15 candidates, totalling 45 fits

Mejores parámetros con SVC : {'class__C': 20, 'class__gamma': 0.01}

Cros-Score con SVC : 0.998



In [21]:
print("Validación: \n", cross_scores_strat)

Validación: 
 {'PCA - RandomForest': 0.9964474634605468, 'Random Forest': 0.999802683504341, 'PCA - SVC': 0.9977363318871452, 'SVC': 0.9977363318871452}
